In [5]:
import cv2
import csv
import numpy as np
import os

image_list = os.listdir('C:/Users/abbya/Downloads/pave_intro-main/images/')
length = len(image_list)
hasCircle = [False]*length
center = [None]*length
radius = [None]*length
distance = [None]*length

for n in range(0, length):
    number = str(n)
    filename = 'C:/Users/abbya/Downloads/pave_intro-main/images/lake'+number+'.jpeg'
    image = cv2.imread(filename)
    
    # detect circle from https://www.geeksforgeeks.org/circle-detection-using-opencv-python/
    # Convert to grayscale.
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Hough transform on the  image.
    detected_circles = cv2.HoughCircles(gray,
        cv2.HOUGH_GRADIENT, 1, 20, param1 = 50,
        param2 = 11, minRadius = 1, maxRadius = 400)

    # Draw circles that are detected.
    if detected_circles is not None:
        hasCircle[n] = True

        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))

        for pt in detected_circles[0]:
            x, y, r = pt[0], pt[1], pt[2]

            # Center, radius, and distance
            center[n] = [x,y]
            
            radius[n] = r
            
            # assumed using camera similar to GoPro as described here: https://improvephotography.com/54797/gopro-good-enough-for-an-advanced-photographers/#:~:text=Compared%20to%20a%20full%20frame,angle%20can%20be%20a%20limitation. 
            # focal length = 3 mm
            focal_length = 3
            # sensor = 6 × 4 mm  
            # number of pixels = 3000 x 2000  
            # height of pixel = 4 / 2000 = 0.002 mm
            pixel_height = 0.002
            # determine image height
            image_height = (2 * r) * pixel_height
            # most popular circular buoy is 15" by 15" (381 mm by 381 mm)
            object_height = 381
            # using ideal pinhole camera model: image size / object size  =  focal length / distance 
            # want distance in meters
            distance_unrounded = (focal_length * object_height / image_height) / 1000
            distance[n] = round(distance_unrounded, 2)
            
            break # so that only take first circle (best one)
            
            
out = open("data.txt", "w")
writer = csv.writer(out)
writer.writerow(center)
writer.writerow(radius)
writer.writerow(distance)
out.close()